# Rest API Serving

## Import Packages

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import load_boston
from sklearn.decomposition import PCA, FastICA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from neuraxle.api.flask import FlaskRestApiWrapper, JSONDataBodyDecoder, JSONDataResponseEncoder
from neuraxle.pipeline import Pipeline
from neuraxle.steps.sklearn import SKLearnWrapper, RidgeModelStacking
from neuraxle.union import AddFeatures

## Load Dataset

In [ ]:
boston = load_boston()
X, y = shuffle(boston.data, boston.target, random_state=13)
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

## Create Pipeline

In [ ]:
pipeline = Pipeline([
    AddFeatures([
        SKLearnWrapper(PCA(n_components=2)),
        SKLearnWrapper(FastICA(n_components=2)),
    ]),
    RidgeModelStacking([
        SKLearnWrapper(GradientBoostingRegressor()),
        SKLearnWrapper(KMeans()),
    ]),
])

## Train

In [ ]:
print("Fitting on train:")
pipeline = pipeline.fit(X_train, y_train)
print("")

print("Transforming train and test:")
y_train_predicted = pipeline.transform(X_train)
y_test_predicted = pipeline.transform(X_test)
print("")

print("Evaluating transformed train:")
score = r2_score(y_train_predicted, y_train)
print('R2 regression score:', score)
print("")

print("Evaluating transformed test:")
score = r2_score(y_test_predicted, y_test)
print('R2 regression score:', score)

## Step to decode json 

CustomJSONDecoderFor2DArray maps the request body json to the expected data inputs format.

In [ ]:
class CustomJSONDecoderFor2DArray(JSONDataBodyDecoder):
    """This is a custom JSON decoder class that precedes the pipeline's transformation."""

    def decode(self, data_inputs):
        """
        Transform json object into an np array.

        :param data_inputs: json object
        :return: np array for data inputs
        """
        return np.array(data_inputs)

## Step to encode json response

CustomJSONEncoderOfOutputs returns a flask Response object that contains the encoded data inputs (predictions).

In [ ]:
class CustomJSONEncoderOfOutputs(JSONDataResponseEncoder):
    """This is a custom JSON response encoder class for converting the pipeline's transformation outputs."""

    def encode(self, data_inputs) -> dict:
        """
        Returns the response dict for the flask Response object.

        :param data_inputs:
        :return:
        """
        return {
            'predictions': list(data_inputs)
        }


## Serve Predictions

FlaskRestApiWrapper will create a flask app that calls the wrapped pipeline transform method on each post request.

In [ ]:
app = FlaskRestApiWrapper(
    json_decoder=CustomJSONDecoderFor2DArray(),
    wrapped=pipeline,
    json_encoder=CustomJSONEncoderOfOutputs(),
    route='/'
).get_app()

app.run(debug=False, port=5000)

## Api Call Example 

In [ ]:
test_predictictions = requests.post(
    url='http://127.0.0.1:5000/',
    json=X_test.tolist()
)
print(test_predictictions)